In [36]:
from scapy.all import *
import sys
import codecs
import re
import numpy as np
from numpy import newaxis
import string
from nltk.corpus import stopwords 
import time
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf
tf.compat.v1.experimental.output_all_intermediates(True)

from joblib import dump, load

from sklearn.metrics import classification_report
from art.attacks.evasion import ProjectedGradientDescent
from art.attacks.evasion import ZooAttack
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import CarliniL2Method
from art.attacks.evasion import SaliencyMapMethod
from art.attacks.evasion import DecisionTreeAttack

import sklearn
from art.estimators import BaseEstimator
from art.estimators.classification import SklearnClassifier
from art.estimators.classification import XGBoostClassifier
from art.estimators.classification import KerasClassifier
from art.estimators.classification import EnsembleClassifier
from art.estimators.classification.scikitlearn import ScikitlearnRandomForestClassifier


from itertools import islice
from tqdm import tqdm
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
tf.compat.v1.disable_eager_execution()


In [2]:
# Import normal Dataset
def import_dataset():
    with open('./Dataset/csv/Original/Attack_merge.csv', newline='') as csvfile:
        rows = pd.read_csv(csvfile,header=None)
        y = rows[10]
        x = rows.drop([10], axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x, y , test_size=0.2)
        y_train_ex = tf.expand_dims(y_train, axis=1)
        X_train_ex = tf.expand_dims(X_train, axis=2)
        y_test_ex = tf.expand_dims(y_test, axis=1)
        X_test_ex = tf.expand_dims(X_test, axis=2)
    return X_train, X_test, y_train, y_test, X_train_ex, X_test_ex, y_train_ex, y_test_ex

In [3]:
# Import Model
def import_model():
    DT = load("./models/DT.joblib")
    RF = load("./models/RF.joblib")
    LR = load("./models/LR.joblib")
    XGB = load("./models/XGB.joblib") 
    SVM = load ("./models/SVM.joblib")
    KNN = load("./models/KNN.joblib")
    DNN = tf.keras.models.load_model("./models/DNN4.h5")
    CNN = tf.keras.models.load_model("./models/CNN.h5")
    LSTM = tf.keras.models.load_model("./models/LSTM.h5")
    return DT, RF, LR, XGB, SVM, KNN, DNN, CNN, LSTM

In [4]:
def DTA(model, model_, X_test):
    print("######## DTA_", model_," ########")
    DT_adv = SklearnClassifier(model)
    attack_DT = DecisionTreeAttack(DT_adv)
    adv = attack_DT.generate(np.array(X_test))
    if not os.path.exists("./Dataset/csv/Attacked"):
        os.mkdir("./Dataset/csv/Attacked")

    adv_i = np.rint(adv[:,1:])
    adv_int = np.insert(adv_i, 0, adv[:,0], axis=1)
    np.savetxt("./Dataset/csv/Attacked/DTA_DT_X_Test.csv", adv_int, delimiter=",")
    return adv

In [24]:
def CW(model, model_, X_test):
    print("######## CW_", model_," ########")
    if model_ =='SVM':
        model_adv = SklearnClassifier(model, clip_values=(0, 10))
    elif model_ == 'LR' or model_ == 'DT':
        model_adv = SklearnClassifier(model)
    elif model_ == 'RF':
        model_adv = ScikitlearnRandomForestClassifier(model)
    elif model_ == 'XGB':
        model_adv = XGBoostClassifier(model=model, nb_features=X_test.shape[1], nb_classes=10)
    else:
        model_adv = KerasClassifier(model, clip_values=(0, 1))

    attack_CW = CarliniL2Method(model_adv, max_iter=20, verbose=True)

    if model_ == "CNN" or model_ =="LSTM":
        X_test = np.array(X_test)
        X_test_ex = X_test[:, :, newaxis]
        adv = attack_CW.generate(np.array(X_test_ex))
    else:
        adv = attack_CW.generate(np.array(X_test))
    if not os.path.exists("./Dataset/csv/Attacked"):
        os.mkdir("./Dataset/csv/Attacked")
    
    adv_i = np.rint(adv[:,1:])
    adv_int = np.insert(adv_i, 0, adv[:,0], axis=1)
    if model_ == "CNN" or model_ =="LSTM":
            np.savetxt("./Dataset/csv/Attacked/CW_" + model_ +"_X_Test.csv", adv_int[:,:,0], delimiter=",")
    else:
        np.savetxt("./Dataset/csv/Attacked/CW_" + model_ +"_X_Test.csv", adv_int, delimiter=",")
    return adv

In [23]:
def JSMA(model, model_, X_test):
    print("######## JSMA_", model_," ########")
    if model_ =='SVM':
        model_adv = SklearnClassifier(model, clip_values=(0, 10))
    elif model_ == 'LR' or model_ == 'DT':
        model_adv = SklearnClassifier(model)
    elif model_ == 'RF':
        model_adv = ScikitlearnRandomForestClassifier(model)
    elif model_ == 'XGB':
        model_adv = XGBoostClassifier(model=model, nb_features=X_test.shape[1], nb_classes=10)
    else:
        model_adv = KerasClassifier(model, clip_values=(0, 1))
        
    attack_JSMA = SaliencyMapMethod(classifier=model_adv)

    if model_ == "CNN" or model_ =="LSTM":
        X_test = np.array(X_test)
        X_test_ex = X_test[:, :, newaxis]
        adv = attack_JSMA.generate(np.array(X_test_ex))
    else:
        adv = attack_JSMA.generate(np.array(X_test))

    if not os.path.exists("./Dataset/csv/Attacked"):
        os.mkdir("./Dataset/csv/Attacked")
    
    adv_i = np.rint(adv[:,1:])
    adv_int = np.insert(adv_i, 0, adv[:,0], axis=1)
    if model_ == "CNN" or model_ =="LSTM":
            np.savetxt("./Dataset/csv/Attacked/JSMA_" + model_ +"_X_Test.csv", adv_int[:,:,0], delimiter=",")
    else:
        np.savetxt("./Dataset/csv/Attacked/JSMA_" + model_ +"_X_Test.csv", adv_int, delimiter=",")
    return adv

In [20]:
def PGD(model, model_, X_test):
    print("######## PGD_", model_," ########")
    if model_ =='SVM':
        model_adv = SklearnClassifier(model, clip_values=(0, 10))
    elif model_ == 'LR' or model_ == 'DT':
        model_adv = SklearnClassifier(model)
    elif model_ == 'RF':
        model_adv = ScikitlearnRandomForestClassifier(model)
    elif model_ == 'XGB':
        model_adv = XGBoostClassifier(model=model, nb_features=X_test.shape[1], nb_classes=10)
    else:
        model_adv = KerasClassifier(model, clip_values=(0, 1))

    attack_PGD = ProjectedGradientDescent(estimator=model_adv, norm=np.inf, eps=.3, eps_step=0.1, max_iter=20, 
                               targeted=False, num_random_init=0, batch_size=128, verbose=False)
    
    if model_ == "CNN" or model_ =="LSTM":
        X_test = np.array(X_test)
        X_test_ex = X_test[:, :, newaxis]
        adv = attack_PGD.generate(np.array(X_test_ex))
    else:
        adv = attack_PGD.generate(np.array(X_test))

    if not os.path.exists("./Dataset/csv/Attacked"):
        os.mkdir("./Dataset/csv/Attacked")
    
    adv_i = np.rint(adv[:,1:])
    adv_int = np.insert(adv_i, 0, adv[:,0], axis=1)
    if model_ == "CNN" or model_ =="LSTM":
        np.savetxt("./Dataset/csv/Attacked/PGD_" + model_ +"_X_Test.csv", adv_int[:,:,0], delimiter=",")
    else:
        np.savetxt("./Dataset/csv/Attacked/PGD_" + model_ +"_X_Test.csv", adv_int, delimiter=",")
    return adv

In [21]:
def ZOO(model, model_, X_test):
    print("######## ZOO_", model_," ########")
    if model_ =='SVM':
        model_adv = SklearnClassifier(model, clip_values=(0, 10))
    elif model_ == 'LR' or model_ == 'DT':
        model_adv = SklearnClassifier(model)
    elif model_ == 'RF':
        model_adv = ScikitlearnRandomForestClassifier(model)
    elif model_ == 'XGB':
        model_adv = XGBoostClassifier(model=model, nb_features=X_test.shape[1], nb_classes=10)
    else:
        model_adv = KerasClassifier(model, clip_values=(0, 1))

    attack_ZOO = ZooAttack(classifier=model_adv, confidence=0.0, targeted=False, learning_rate=1e-1, max_iter=20,
                    binary_search_steps=10, initial_const=1e-3, abort_early=True, use_resize=False, 
                    use_importance=False, nb_parallel=1, batch_size=1, variable_h=0.2)
    
    if model_ == "CNN" or model_ =="LSTM":
        X_test = np.array(X_test)
        X_test_ex = X_test[:, :, newaxis]
        adv = attack_ZOO.generate(np.array(X_test_ex))
    else:
        adv = attack_ZOO.generate(np.array(X_test))

    if not os.path.exists("./Dataset/csv/Attacked"):
        os.mkdir("./Dataset/csv/Attacked")
    
    adv_i = np.rint(adv[:,1:])
    adv_int = np.insert(adv_i, 0, adv[:,0], axis=1)
    if model_ == "CNN" or model_ =="LSTM":
        np.savetxt("./Dataset/csv/Attacked/ZOO_" + model_ +"_X_Test.csv", adv_int[:,:,0], delimiter=",")
    else:
        np.savetxt("./Dataset/csv/Attacked/ZOO_" + model_ +"_X_Test.csv", adv_int, delimiter=",")
    return adv

In [38]:
def FGSM(model, model_, X_test):
    print("######## FGSM_", model_," ########")
    if model_ =='SVM' or model_ == "RF" :
        model_adv = SklearnClassifier(model, clip_values=(0, 10))
    elif model_ == 'LR':
        model_adv = SklearnClassifier(model)
    elif model_ == 'RF':
        model_adv = ScikitlearnRandomForestClassifier(model)
    elif model_ == 'XGB':
        model_adv = XGBoostClassifier(model)
    elif model_ == 'KNN':
        model_adv = BaseEstimator(model)
    else:
        model_adv = KerasClassifier(model, clip_values=(0, 1))

    attack_FGSM = FastGradientMethod(estimator=model_adv, eps=0.3)
    if model_ == "CNN" or model_ == "LSTM":
        X_test = np.array(X_test)
        X_test_ex = X_test[:, :, newaxis]
        adv = attack_FGSM.generate(np.array(X_test_ex))
    else:
        adv = attack_FGSM.generate(np.array(X_test))

    if not os.path.exists("./Dataset/csv/Attacked"):
        os.mkdir("./Dataset/csv/Attacked")
    
    adv_i = np.rint(adv[:,1:])
    adv_int = np.insert(adv_i, 0, adv[:,0], axis=1)
    if model_ == "CNN" or model_ =="LSTM":
        np.savetxt("./Dataset/csv/Attacked/FGSM_" + model_ +"_X_Test.csv", adv_int[:,:,0], delimiter=",")
    else:
        np.savetxt("./Dataset/csv/Attacked/FGSM_" + model_ +"_X_Test.csv", adv_int, delimiter=",")
    return adv

In [10]:
X_train, X_test, y_train, y_test, X_train_ex, X_test_ex, y_train_ex, y_test_ex = import_dataset()
DT, RF, LR, XGB, SVM, KNN, DNN, CNN, LSTM = import_model()
#DTA FGSM(完成 快) CW(很久) JSMA(跑條跑不出來) PGD(完成) ZOO(很久 ing)

Instructions for updating:
Colocations handled automatically by placer.


In [37]:
attack_FGSM = SaliencyMapMethod(classifier=BaseEstimator(KNN))

TypeError: Can't instantiate abstract class BaseEstimator with abstract methods fit, input_shape, predict

In [39]:
# FGSM(LR, "LR", X_test) OK
#FGSM(XGB, "XGB", X_test)
# FGSM(RF, "RF", X_test)
#FGSM(SVM, "SVM", X_test) OK
FGSM(KNN, "KNN", X_test)
#FGSM(DNN, "DNN", X_test) OK
#FGSM(CNN, "CNN", X_test) OK
# FGSM(LSTM, "LSTM", X_test)

######## FGSM_ KNN  ########


TypeError: Can't instantiate abstract class BaseEstimator with abstract methods fit, input_shape, predict

In [ ]:
#CW(LR, "LR", X_test) OK
#CW(SVM, "SVM", X_test) OK 
#CW(KNN, "KNN", X_test) ?
#CW(DNN, "DNN", X_test) NO

In [ ]:
#ZOO(DT, "DT", X_test) OK
#ZOO(RF, "RF", X_test) OK
#ZOO(LR, "LR", X_test) OK
#ZOO(XGB, "XGB", X_test) OK
#ZOO(SVM, "SVM", X_test) OK
ZOO(KNN, "KNN", X_test)
#ZOO(DNN, "DNN", X_test) OK
ZOO(CNN, "CNN", X_test)
ZOO(LSTM, "LSTM", X_test)

In [18]:
#JSMA(LR, "LR", X_test) 
#JSMA(SVM, "SVM", X_test) 
#JSMA(KNN, "KNN", X_test) ?
#JSMA(DNN, "DNN", X_test)
JSMA(CNN, "CNN", X_test)
JSMA(LSTM, "LSTM", X_test)

######## JSMA_ LSTM  ########


JSMA:   0%|          | 0/44730 [00:00<?, ?it/s]


IndexError: list index out of range

In [25]:
# PGD(LR, "LR", X_test) 
# PGD(SVM, "SVM", X_test) 
#PGD(KNN, "KNN", X_test) ?
# PGD(DNN, "DNN", X_test)
PGD(CNN, "CNN", X_test)
PGD(LSTM, "LSTM", X_test)

######## PGD_ CNN  ########
######## PGD_ LSTM  ########


array([[[0.0000000e+00],
        [3.2970001e+02],
        [1.0000000e-01],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00]],

       [[2.0517811e-01],
        [3.2970001e+02],
        [1.2700000e+01],
        ...,
        [1.2700000e+01],
        [1.7070000e+02],
        [0.0000000e+00]],

       [[2.0531112e-01],
        [3.2970001e+02],
        [1.2700000e+01],
        ...,
        [1.2700000e+01],
        [1.7070000e+02],
        [0.0000000e+00]],

       ...,

       [[3.0530006e-01],
        [4.8670001e+02],
        [1.0000000e-01],
        ...,
        [0.0000000e+00],
        [3.0000001e-01],
        [3.0000001e-01]],

       [[3.0000186e-01],
        [7.7670001e+02],
        [3.2700001e+01],
        ...,
        [1.0000000e-01],
        [1.0000000e-01],
        [1.0000000e-01]],

       [[0.0000000e+00],
        [3.2970001e+02],
        [1.2700000e+01],
        ...,
        [1.2700000e+01],
        [1.7070000e+02],
        [3.0000001e-01]]

In [ ]:
adv = DTA(DT, "DT", X_test)

c:\Users\Maxwang\anaconda3\envs\Adversarial_attack_\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
Decision tree attack: 100%|██████████| 44730/44730 [00:17<00:00, 2615.59it/s]


In [ ]:
adv_i = np.rint(adv[:,1:])
arr = np.insert(adv_i, 0, adv[:,0], axis=1)
np.savetxt("./Dataset/csv/Attacked/test.csv", arr, delimiter=",")

In [ ]:
model = DT
model.fit(X_train,y_train)
print(model.score(X_test, y_test))
prediction = model.predict(X_test)
print(classification_report(y_test,prediction))

0.971070869662419
              precision    recall  f1-score   support

           0       0.97      0.98      0.97     23042
           1       0.97      0.97      0.97     21688

    accuracy                           0.97     44730
   macro avg       0.97      0.97      0.97     44730
weighted avg       0.97      0.97      0.97     44730



In [ ]:
model = DT
model.fit(X_train,y_train)
print(model.score(arr, y_test))
prediction = model.predict(arr)
print(classification_report(y_test,prediction))

0.44804381846635366
              precision    recall  f1-score   support

           0       0.33      0.07      0.12     23042
           1       0.46      0.85      0.60     21688

    accuracy                           0.45     44730
   macro avg       0.40      0.46      0.36     44730
weighted avg       0.39      0.45      0.35     44730

